# Encodings creation notebook
This notebook was used to get the encodings of the images using a VGG19

In [ ]:
import json, os
import random
from matplotlib.image import imread
import numpy as np
import tensorflow as tf
from PIL import Image

Reading dataset json files

In [ ]:
img_h = 224
img_w = 224


cwd = os.getcwd()
datasetName = '../input/ann-and-dl-vqa/dataset_vqa'
jsonFiles = '../input/json-files'
trainJsonName = 'train_data.json'
validJsonName = 'valid_data.json'
imagesPath = os.path.join(datasetName, 'train')
trainJsonPath = os.path.join(jsonFiles, trainJsonName)
validJsonPath = os.path.join(jsonFiles, validJsonName)

with open(trainJsonPath,'r') as json_file_train, open (validJsonPath, 'r') as json_file_valid:
    data_train = json.load(json_file_train).get('questions')
    data_valid = json.load(json_file_valid).get('questions')
    json_file_train.close()
    json_file_valid.close()
    
print(data_train[0])

Instantiating the encoder structure

In [ ]:
fe = tf.keras.applications.VGG19(include_top = False, pooling = 'avg', input_shape = (img_h,img_w,3), weights = 'imagenet')
for l in fe.layers:
    l.trainable = False

model = tf.keras.Sequential()
model.add(fe)

model.summary()

### Get the encoding of the training and validation dataset

In [ ]:
train_images = set([el['image_filename'] for el in data_train])
valid_images = set([el['image_filename'] for el in data_valid])
len_train = len(train_images)
len_valid = len(valid_images)
print('taken the ' + str(len(valid_images)/len(train_images))+' of validation')

In [ ]:
train_tensor_map = {}
i = 0
for imagename in train_images:
    print('{:3.2f} %'.format(i/len_train * 100), end = '\r')
    image = Image.open(os.path.join(imagesPath, imagename)).resize((img_w, img_h)).convert('RGB')
    img = np.array(image).astype(np.float32) / 255
    res = model.predict(x = np.expand_dims(img,0))
    train_tensor_map[str(imagename)] = res.tolist()
    i = i + 1

json.dump(train_tensor_map, open("train_tensors_VGG19_GAP.json","w"), indent=2)

In [ ]:
valid_tensor_map = {}
i = 0
for imagename in valid_images:
    print('{:3.2f} %'.format(i/len_valid * 100), end = '\r')
    image = Image.open(os.path.join(imagesPath, imagename)).resize((img_w, img_h)).convert('RGB')
    img = np.array(image).astype(np.float32) / 255
    res = model.predict(x = np.expand_dims(img,0))
    valid_tensor_map[str(imagename)] = res.tolist()
    i = i + 1
    
json.dump(valid_tensor_map, open("valid_tensors_VGG19_GAP.json","w"), indent=2)

### Get the encoding of the test data

In [ ]:
testJsonName = 'test_data.json'
imagesPathTest = os.path.join(datasetName,'test')
testJsonPath = os.path.join(datasetName, testJsonName)

with open (testJsonPath, 'r') as json_file_test:
    data_test = json.load(json_file_test).get('questions')
    json_file_test.close()
    
test_images = set([el['image_filename'] for el in data_test])

len_test = len(test_images)

test_tensor_map = {}
i = 0
for imagename in test_images:
    print('{:3.2f} %'.format(i/len_test * 100),end = '\r')
    image = Image.open(os.path.join(imagesPathTest, imagename)).resize((img_w, img_h)).convert('RGB')
    img = np.array(image).astype(np.float32) / 255
    res = model.predict(x = np.expand_dims(img,0))
    test_tensor_map[str(imagename)] = res.tolist()
    i = i + 1

json.dump(test_tensor_map, open("test_tensors_VGG19_GAP.json","w"), indent=2)
